In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import math

# -------------------------------------------------
# SABİT VERİ (E-n13-k4) – TSPLIB’den gömülü alındı
# -------------------------------------------------
n = 13
depot = 0
K = 4
C = 6000
demand = [0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700, 1100]

numbers = [
     9, 14, 21, 23, 22, 25, 32, 36, 38, 42, 50, 52,
     5, 12, 22, 21, 24, 31, 35, 37, 41, 49, 51,
     7, 17, 16, 23, 26, 30, 36, 36, 44, 46,
     10, 21, 30, 27, 37, 43, 31, 37, 39,
     19, 28, 25, 35, 41, 29, 31,
     29,  9, 10, 16, 22, 20, 28,
     30,  7, 11, 13, 17, 25,
     27, 10, 16, 10, 18,
     20,  6,  6, 14,
     16, 12, 12, 20,
      8, 10, 10
]

def build_cost_from_lower_row(numbers, n):
    cost = [[0]*n for _ in range(n)]
    idx = 0
    for i in range(2, n+1):
        for j in range(1, i):
            cij = numbers[idx]
            idx += 1
            cost[i-1][j-1] = cij
            cost[j-1][i-1] = cij
    return cost

cost = build_cost_from_lower_row(numbers, n)

# -------------------------------------------------
# OR-Tools ile CVRP çözümü
# -------------------------------------------------
def solve_cvrp_ortools(cost, demand, C, K, depot=0):
    n = len(demand)
    # Index Manager
    manager = pywrapcp.RoutingIndexManager(n, K, depot)
    routing = pywrapcp.RoutingModel(manager)

    # Distance callback
    def distance_callback(from_index, to_index):
        return cost[manager.IndexToNode(from_index)][manager.IndexToNode(to_index)]
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Capacity callback
    def demand_callback(from_index):
        return demand[manager.IndexToNode(from_index)]
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # slack
        [C]*K,  # vehicle capacities
        True,
        "Capacity"
    )

    # Search parameters
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        routes = []
        total_distance = 0
        for v in range(K):
            index = routing.Start(v)
            route = [depot]
            route_distance = 0
            route_load = 0
            while not routing.IsEnd(index):
                node = manager.IndexToNode(index)
                next_index = solution.Value(routing.NextVar(index))
                next_node = manager.IndexToNode(next_index) if not routing.IsEnd(next_index) else depot
                route_distance += cost[node][next_node]
                if node != depot:
                    route_load += demand[node]
                index = next_index
                if node != depot:
                    route.append(node)
            if len(route) > 1:
                route.append(depot)
                routes.append((route, route_load, route_distance))
                total_distance += route_distance

        # Print results
        for i, (r, load, dist) in enumerate(routes, 1):
            print(f"Route {i}: {r} | load={load} | dist={dist}")
        print(f"Total distance: {total_distance}")
        return routes, total_distance
    else:
        print("No solution found.")
        return None, None

# -------------------------------------------------
# Çalıştır
# -------------------------------------------------
routes, total_distance = solve_cvrp_ortools(cost, demand, C, K, depot)
